# Recommendation Engines: Implementing Surprise
- Surprise is a scikit library that stands for **Simple Python RecommendatIon System Engine**
- Has built-in similarity metrics, baseline methods, content-based systems, matrix factorization systems

In this notebook, we'll first walk through setting up a super basic recommendation system, using the popular MovieLens 100K Dataset. Then, we'll look into more detail how Surprise works.

## Fitting and Predicting with Surprise

### 1. Install surprise if you haven't, and import the usual libraries.

In [1]:
!pip install surprise

     |████████████████████████████████| 6.5MB 2.2MB/s eta 0:00:01
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp37-cp37m-macosx_10_9_x86_64.whl size=799586 sha256=e2c3a80238240060c74c7e5e974c59d0c8d985230fe8e071b555e6eff740322e
  Stored in directory: /Users/robzifchak/Library/Caches/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [ ]:
# import libraries
import numpy as np
import pandas as pd

from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split

### 2. Load in the dataset

Surprise has the dataset built in. You might need to download the dataset so follow the instructions in the code output! Unfortunately, the Surprise data format doesn't let us inspect the data, but here is the documentation: https://grouplens.org/datasets/movielens/100k/


In [ ]:
data = Dataset.load_builtin('ml-100k')

# train-test split
train, test = train_test_split(data, test_size=.2)

In [ ]:
train

### 3. Run the default Singular Value Decomposition Model!

In [ ]:
svd = SVD()
svd.fit(train)
predictions = svd.test(test)

In [ ]:
accuracy.rmse(predictions)

### 4. Make a prediction!

In [ ]:
uid = str(196)  # raw user id (as in the ratings file). They are **strings**!
iid = str(302)

# get a prediction for specific users and items.
pred = svd.predict(uid, iid, r_ui=4, verbose=True)

## Applying Surprise

### 1. How does Surprise take in your data?
https://surprise.readthedocs.io/en/stable/getting_started.html#use-a-custom-dataset

The dataset we'll use is a subset of the Yelp Open Dataset that's already been joined and cleaned.
https://www.yelp.com/dataset

In [ ]:
yelp = pd.read_csv('yelp_reviews.csv').drop(['Unnamed: 0'], axis = 1)

In [ ]:
yelp.head()

### 2. Inspecting the dataset:

Here's where you'd do a **comprehensive** EDA!

In [ ]:
print('Number of Users: ', len(yelp['user_id'].unique()))
print('Number of Businesses: ', len(yelp['business_id'].unique()))

1. What's the distribution of ratings? i.e. How many 1-star, 2-star, 3-star reviews?
2. How many reviews does a restaurant have?
3. How many reviews does a user make?

In [ ]:
yelp['stars'].value_counts()

In [ ]:
yelp['business_id'].value_counts()

### 3. Reading in the dataset and prepping data

In [ ]:
# Instantiate a 'Reader' to read in the data so Surprise can use it
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(yelp[['user_id', 'business_id', 'stars']], reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=.2)

In [ ]:
trainset

### 4. Fitting and evaluating models
Here, let's assume that we've tuned all these hyperparameters using GridSearch, and we've arrived at our final model.

In [ ]:
final = SVD(n_epochs=20, n_factors=1, biased=True, 
              lr_all=0.005, reg_all=0.06)

In [ ]:
final.fit(trainset)

In [ ]:
predictions = final.test(testset)

In [ ]:
predictions[:3]

In [ ]:
accuracy.rmse(predictions)
accuracy.mae(predictions)

### 5. Making Predictions (again)
Unfortunately, this dataset has a convoluted string as the user/business IDs.

In [ ]:
yelp['user_id'][55]

In [ ]:
yelp['business_id'][123]

In [ ]:
final.predict(yelp['user_id'][55], yelp['business_id'][13])

### 6. What else?

Surprise has sample code where you can get the top **n** recommended items for a user. https://surprise.readthedocs.io/en/stable/FAQ.html

# Resources

- The structure of our lesson on recommendation engines is based on Chapter 9 of **Mining of Massive Datasets**: http://infolab.stanford.edu/~ullman/mmds/book.pdf
- Libraries for coding recommendation engines: 
    - Surprise: https://surprise.readthedocs.io/en/stable/index.html
    - LightFM: https://lyst.github.io/lightfm/docs/index.html
    
    
- Some blogs I might've written:
    - Overview: https://towardsdatascience.com/a-primer-to-recommendation-engines-49bd12ed849f?source=friends_link&sk=279dfeec5187614b37431dab167fd4e3
    - Collaborative filtering: https://towardsdatascience.com/a-primer-to-recommendation-engines-49bd12ed849f?source=friends_link&sk=279dfeec5187614b37431dab167fd4e3